<a href="https://colab.research.google.com/github/panaku88/MCS-7103-Machine-Learning/blob/main/EDA_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
raw_dataset = pd.read_csv('/content/drive/MyDrive/MCSC1/dataset/CS_Service_Data.csv')
#raw_dataset.head()

In [53]:
import re
import random
# Replace 'RKE' with 'TT' in the 'TICKET NUMBER' column
raw_dataset['TICKET NUMBER'] = raw_dataset['TICKET NUMBER'].str.replace(r'^RKE', 'TT', regex=True)

# Define a function to replace 'RKE' with 'TT' in any word
def replace_rke_with_tt(text):
    # Check if the value is a string
    if isinstance(text, str):
        # Replace any word starting with 'RKE' with 'TT' but keep the rest of the word unchanged
        return re.sub(r'\bRKE(\w*)\b', r'TT\1', text)
    else:
        # Return the original value if it's not a string
        return text


def replace_service_plan(text):
  if isinstance(text, str):
    if re.match(r'CAPPED-BASE: Roke Capped Base', text):
      return 'SONIC HOME PRO 25Mbps'
    else:
      return text
  else:
    return text

def replace_service_plan_ent(text):
  if isinstance(text, str):
    match = re.search(r'RE(\d+): Roke Enterprise', text)
    if match:
      number = match.group(1)
      return f'SONIC BUSINESS {number}Mbps'
    else:
      return text
  else:
    return text


def replace_service_plan_vpn(text):
  if isinstance(text, str):
    if 'VPN' in text or 'vpn' in text:
      random_number = random.randint(1, 100)
      return f'SONIC MPLS VPN {random_number}Mbps'
    else:
      return text
  else:
    return text

raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan_vpn)
raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan_ent)
raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan)
raw_dataset['STATUS DESCRIPTION'] = raw_dataset['STATUS DESCRIPTION'].apply(replace_rke_with_tt)

# Specify the path to save the modified dataset
output_path = '/content/drive/MyDrive/MCSC1/dataset/clean_cs_dataset.csv'



In [54]:
customer_names = {}
syllables = ['a', 'e', 'i', 'o', 'u', 'ka', 'ke', 'ki', 'ko', 'ku', 'sa', 'se', 'si', 'so', 'su', 'ta', 'te', 'ti', 'to', 'tu', 'na', 'ne', 'ni', 'no', 'nu', 'ha', 'he', 'hi', 'ho', 'hu', 'ma', 'me', 'mi', 'mo', 'mu', 'ya', 'ye', 'yi', 'yo', 'yu', 'ra', 're', 'ri', 'ro', 'ru', 'wa', 'we', 'wi', 'wo', 'wu']

def generate_name(min_length=3, max_length=6):
  name = ''
  length = random.randint(min_length, max_length)
  for i in range(length):
    name += random.choice(syllables)
  return name.capitalize()

def generate_customer_name(row):
  name = row['CUSTOMER NAME']
  account = row['CUSTOMER ACCOUNT']
  if isinstance(name, str):
    if account in customer_names:
      return customer_names[account]
    else:
      if 'Mr' in name or 'Ms' in name:
        title = random.choice(['Mr', 'Ms'])
        first_name = generate_name()
        last_name = generate_name()
        new_name = f'{title} {first_name} {last_name}'
      elif 'Company' in name or 'Ltd' in name or 'Inc' in name:
        new_name = generate_name() + ' Inc'
      else:
        first_name = generate_name()
        last_name = generate_name()
        new_name = f'{first_name} {last_name}'
      customer_names[account] = new_name
      return new_name
  else:
    return name

raw_dataset['CUSTOMER NAME'] = raw_dataset.apply(generate_customer_name, axis=1)

In [55]:
def replace_site_id_description(row):
  site_id = row['SITE  ID']
  site_description = row['SITE DESCRIPTION']
  if isinstance(site_id, str) and isinstance(site_description, str):
    if 'BCS' in site_id or 'BCS' in site_description:
      return 'SONIC GPON'
    else:
      return site_id
  else:
    return site_id

raw_dataset['SITE  ID'] = raw_dataset.apply(replace_site_id_description, axis=1)
raw_dataset['SITE DESCRIPTION'] = raw_dataset.apply(replace_site_id_description, axis=1)

In [56]:
# Save the modified dataset to the specified path
raw_dataset.to_csv(output_path, index=False)

print(f"The clean_cs_dataset.csv has been created successfully at {output_path}.")

The clean_cs_dataset.csv has been created successfully at /content/drive/MyDrive/MCSC1/dataset/clean_cs_dataset.csv.


In [57]:
# Load the dataframe.
new_dataset = pd.read_csv('/content/drive/MyDrive/MCSC1/dataset/clean_cs_dataset.csv')

# Drop rows with 'Not Specified' or 'Not Selected'.
new_dataset = new_dataset[new_dataset.applymap(lambda x: 'Not Specified' not in str(x) and 'Not Selected' not in str(x)).all(axis=1)]
# Drop the columns.
new_dataset = new_dataset.drop(['SITE  ID', 'SITE DESCRIPTION', 'CUSTOMER NAME'], axis=1)
# Save the modified dataset to the specified path
output_path = '/content/drive/MyDrive/MCSC1/dataset/clean_cs_dataset.csv'
new_dataset.to_csv(output_path, index=False)

print(f"The clean_cs_dataset.csv has been created successfully at {output_path}.")

<ipython-input-57-5a42b6c4b393>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  new_dataset = new_dataset[new_dataset.applymap(lambda x: 'Not Specified' not in str(x) and 'Not Selected' not in str(x)).all(axis=1)]


The clean_cs_dataset.csv has been created successfully at /content/drive/MyDrive/MCSC1/dataset/clean_cs_dataset.csv.


In [ ]:
new_dataset.head()

In [ ]:
new_dataset.info()

In [ ]:
new_dataset.tail()

In [ ]:
new_dataset.describe()

In [64]:
new_dataset.isnull().sum()

,0
TICKET NUMBER,0
CUSTOMER ACCOUNT,0
SERVICE PLAN,0
SOURCE,1
Logged By User,0
DATE LOGGED,0
CATEGORY,212
SUB-CATEGORY,212
PRIORITY,0
LOGGED DESCRIPTION,0


In [ ]:
new_dataset.shape

(252608, 24)

In [ ]:
new_dataset.columns

In [ ]:
new_dataset.dtypes

In [ ]:
new_dataset.nunique()

In [ ]:
new_dataset.duplicated().sum()

0

In [ ]:
new_dataset

In [ ]:
# Histograms
new_dataset.hist(figsize=(10,10))
plt.show()

In [ ]:
# Box plots
new_dataset.plot(kind='box', subplots=True, layout=(4,4), figsize=(10,10))
plt.show()

In [ ]:
# Scatter plots
plt.scatter(new_dataset['DATE LOGGED'], new_dataset['TIME IDLE (MINS)'])
plt.xlabel('column1')
plt.ylabel('column2')
plt.show()


In [ ]:
# Correlation Analysis
correlation_matrix = new_dataset.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:

# Identify Outliers (using box plots - refer to box plot code above)

# Groupby and Aggregation
new_dataset.groupby('DATE LOGGED')['TIME IDLE (MINS)'].sum()